In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Text Embedding New API

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/text_embedding_new_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/generative_ai/text_embedding_new_api.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/generative_ai/text_embedding_new_api.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      Open in Vertex AI Workbench
    </a>
  </td>                                                                                               
</table>

**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook is a code example for how to call our newly released text emebedding models (text-embedding-preview-0409 and text-multilingual-embedding-preview-0409).

Learn more about [text embedding api](https://cloud.google.com/vertex-ai/docs/generative-ai/embeddings/get-text-embeddings#api_changes_to_models_released_in_or_after_august_2023).

### Objective

In this tutorial, you learn how to call text embedding latest APIs on two
new models, text-embedding-preview-0409 and text-multilingual-embedding-preview-0409:

This tutorial uses the following Google Cloud ML services and resources:

- Vertex LLM SDK


The steps performed include:

- Installation and imports
- Generate embeddings

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing),
and use the [Pricing Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Installation

Install the following packages required to execute this notebook.

In [ ]:
# Install Vertex AI SDK for Python
!pip3 install --quiet --upgrade google-cloud-aiplatform

### Colab only: Uncomment the following cell to restart the kernel.

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

## Before you begin

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

3. [Enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). {TODO: Update the APIs needed for your tutorial. Edit the API names, and update the link to append the API IDs, separating each one with a comma. For example, container.googleapis.com,cloudbuild.googleapis.com}

4. If you are running this notebook locally, you need to install the [Cloud SDK](https://cloud.google.com/sdk).

#### Set your project ID

**If you don't know your project ID**, try the following:
* Run `gcloud config list`.
* Run `gcloud projects list`.
* See the support page: [Locate the project ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "sc4052"  # @param {type:"string"}

!gcloud config set project {PROJECT_ID}

Updated property [core/project].


#### Set your region

You can also set the `REGION` variable to use with Vertex AI. Learn more about [Vertex AI regions](https://cloud.google.com/vertex-ai/docs/general/locations).

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

### Authenticate your Google Cloud account

Depending on your Jupyter environment, you may have to manually authenticate. Follow the relevant instructions below.

**1. Vertex AI Workbench**
* Do nothing as you are already authenticated.

**2. Local JupyterLab instance, uncomment and run:**

In [ ]:
!gcloud auth login

Go to the following link in your browser:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=RCRwN1ruqke30ENQGBGplPgDwATo5D&prompt=consent&token_usage=remote&access_type=offline&code_challenge=BkEUA0Ochybi0maqIJGiIhb5BeyHPT71kyJsvtkGEMo&code_challenge_method=S256

Enter authorization code: 

Command killed by keyboard interrupt

^C


**3. Colab, uncomment and run:**

In [ ]:
from google.colab import auth
auth.authenticate_user(project_id=PROJECT_ID)

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project and region.

In [ ]:
import vertexai

vertexai.init(project=PROJECT_ID, location=REGION)

### Generate embeddings

1.   Set the model name. The latest models are
     * "text-embedding-preview-0409" for English.
     * "text-multilingual-embedding-preview-0409" for i18n. See the [language coverage](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#language_coverage_for_textembedding-gecko-multilingual_models) for the list of supported languages.
     
     See [the public doc](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings) for the full list of supported models.
2.   Set the task type, text, and title (*optional and valid only for task type "RETRIEVAL_DOCUMENT"*). The valid task types are:
     * "RETRIEVAL_QUERY"
     * "RETRIEVAL_DOCUMENT"
     * "SEMANTIC_SIMILARITY"
     * "CLASSIFICATION"
     * "CLUSTERING"
     * "QUESTION_ANSWERING" (*valid only for the latest models*)
     * "FACT_VERIFICATION" (*valid only for the latest models*)
3. Set the output dimensionality (*optional and valid only for the latest models*).

In [ ]:
# @title { run: "auto" }
MODEL = "textembedding-gecko@003"  # @param ["text-embedding-preview-0409", "text-multilingual-embedding-preview-0409", "textembedding-gecko@003", "textembedding-gecko-multilingual@001"]
TASK = "RETRIEVAL_DOCUMENT"  # @param ["RETRIEVAL_QUERY", "RETRIEVAL_DOCUMENT", "SEMANTIC_SIMILARITY", "CLASSIFICATION", "CLUSTERING", "QUESTION_ANSWERING", "FACT_VERIFICATION"]
TEXT = "NTU Administered Bursary consists of NTU, Higher Education, Donated and Higher Education Community Bursary.\\n\\nTo be eligible for a bursary, gross monthly Per Capita Income (PCI) of less than or equal to $2,250 or Gross monthly Household Income (GHI) of less than or equal to $9,000.\\n\\nEligible applicants are accorded a bursary based on their financial situation and the criteria set down by the University / Donor.\\n\\nFull Time Undergraduate\\n\\nThe NTU-administered bursaries are open to Singaporean students.\\n\\nSingapore PR and international students may be considered for bursaries sponsored by private individuals and organizations. The number of bursaries available to Singapore PR and International students is limited.\\n\\nPart Time Undergraduate\\n\\nOnly Singaporean students pursuing their part-time undergraduate degree course and receiving MOE Subsidy are eligible.\\n\\nPart-time undergrad students taking only repeat modules in the semester are not eligible for the bursary for that semester.\\n\\nFull-time SPR/International students paying non-subsidised tuition fees are not eligible to apply.\\n\\nNIE trainee teachers who are drawing salary/stipend and whose tuition fees are paid by MOE during the course of study are not eligible to apply."  # @param {type:"string"}
TITLE = ""  # @param {type:"string"}
OUTPUT_DIMENSIONALITY = None  # @param [1, 768, "None"] {type:"raw", allow-input:true}

if not MODEL:
    raise ValueError("MODEL must be specified.")
if not TEXT:
    raise ValueError("TEXT must be specified.")
if TITLE and TASK != "RETRIEVAL_DOCUMENT":
    raise ValueError("TITLE can only be specified for TASK 'RETRIEVAL_DOCUMENT'")
if OUTPUT_DIMENSIONALITY is not None and MODEL not in [
    "text-embedding-preview-0409",
    "text-multilingual-embedding-preview-0409",
]:
    raise ValueError(f"OUTPUT_DIMENTIONALITY cannot be specified for model '{MODEL}'.")
if TASK in ["QUESTION_ANSWERING", "FACT_VERIFICATION"] and MODEL not in [
    "text-embedding-preview-0409",
    "text-multilingual-embedding-preview-0409",
]:
    raise ValueError(f"TASK '{TASK}' is not valid for model '{MODEL}'.")

In [ ]:
import json



# # Rename functions to info1, info2, info3, etc.
# for i, func in enumerate(function_data_adjusted, 1):
#     func['name'] = f'info{i}'

# # Serialize the updated dictionary to a JSON formatted string again to maintain structure
# final_renamed_json_str = json.dumps(function_data_adjusted, indent=2)

# # Path for the renamed JSON file
# final_renamed_file_path = 'final_renamed_data.json'

# # Save the renamed JSON string to a file
# with open(final_renamed_file_path, 'w') as renamed_file:
#     renamed_file.write(final_renamed_json_str)

# final_renamed_file_path


from vertexai.language_models import TextEmbeddingInput, TextEmbeddingModel


def embed_text(
    model_name: str,
    task_type: str,
    text: str,
    title: str = "",
    output_dimensionality=None,
) -> list:
    """Generates a text embedding with a Large Language Model."""
    model = TextEmbeddingModel.from_pretrained(model_name)
    text_embedding_input = TextEmbeddingInput(
        task_type=task_type, title=title, text=text
    )
    kwargs = (
        dict(output_dimensionality=output_dimensionality)
        if output_dimensionality
        else {}
    )
    embeddings = model.get_embeddings([text_embedding_input], **kwargs)
    return embeddings[0].values

# Read from JSON file
with open('/content/complete_transformed_with_system_message.json') as file:
    function_data_adjusted = json.load(file)

MODEL = "textembedding-gecko@003"
TASK = "RETRIEVAL_DOCUMENT"
TEXT = "NTU Administered Bursary consists of NTU, Higher Education, Donated and Higher Education Community Bursary.\\n\\nTo be eligible for a bursary, gross monthly Per Capita Income (PCI) of less than or equal to $2,250 or Gross monthly Household Income (GHI) of less than or equal to $9,000.\\n\\nEligible applicants are accorded a bursary based on their financial situation and the criteria set down by the University / Donor.\\n\\nFull Time Undergraduate\\n\\nThe NTU-administered bursaries are open to Singaporean students.\\n\\nSingapore PR and international students may be considered for bursaries sponsored by private individuals and organizations. The number of bursaries available to Singapore PR and International students is limited.\\n\\nPart Time Undergraduate\\n\\nOnly Singaporean students pursuing their part-time undergraduate degree course and receiving MOE Subsidy are eligible.\\n\\nPart-time undergrad students taking only repeat modules in the semester are not eligible for the bursary for that semester.\\n\\nFull-time SPR/International students paying non-subsidised tuition fees are not eligible to apply.\\n\\nNIE trainee teachers who are drawing salary/stipend and whose tuition fees are paid by MOE during the course of study are not eligible to apply."  # @param {type:"string"}
TITLE = ""
OUTPUT_DIMENSIONALITY = None

count = 0
output_json = []
for item in function_data_adjusted:
    for message in item["messages"]:
        if message["role"] == "assistant":
            count += 1
            TEXT = message["content"]
            embedding = embed_text(
                model_name=MODEL,
                task_type=TASK,
                text=TEXT,
                title=TITLE,
                output_dimensionality=OUTPUT_DIMENSIONALITY,
            )
            output_json.append({
                "id": f"info{count}",
                "name" : f"info{count}",
                "embedding": embedding

            })
# Save the output_json as a JSONL file
with open("output.jsonl", "w") as f:
    for item in output_json:
        json_line = json.dumps(item)
        f.write(json_line + "\n")

print("Output JSONL saved to output.jsonl")

# Get a text embedding for a downstream task.
# embedding = embed_text(
#     model_name=MODEL,
#     task_type=TASK,
#     text=TEXT,
#     title=TITLE,
#     output_dimensionality=OUTPUT_DIMENSIONALITY,
# )
print(len(embedding))  # Expected value: {OUTPUT_DIMENSIONALITY}.

Output JSONL saved to output.jsonl
768


In [ ]:
import json
# Opening JSON file
file1 = open('final_output.json')

# returns JSON object as
# a dictionary
data1 = json.load(file1)

# Iterating through the json
# list
for i in range(0,10):
    print(data1[i])

# Closing file
# f.close()

{'id': 'info1', 'name': 'info1', 'embedding': [-0.005074364133179188, -0.0917763039469719, -0.062171004712581635, 0.03133784979581833, 0.09559568017721176, 0.012958256527781487, 0.01822054572403431, -0.017114099115133286, -0.005451129283756018, 0.023461533710360527, -0.03360050916671753, 0.008128435350954533, -0.048076704144477844, -0.03164680674672127, -0.02820366621017456, -0.036191146820783615, -0.0026136706583201885, 0.03538517653942108, 0.002464496763423085, -0.050866901874542236, -0.0009600920602679253, 0.024141812697052956, -0.01119875255972147, -0.043478965759277344, -0.0032314511481672525, -0.003984016831964254, -0.02441326715052128, -0.05744209513068199, -0.053170833736658096, 0.020085496827960014, -0.050620321184396744, 0.05556102842092514, -0.01726190187036991, 0.026790235191583633, -0.0057145943865180016, -0.040730107575654984, 0.005809189286082983, -0.030071722343564034, -0.039063867181539536, 0.009377000853419304, -0.0014375814935192466, -0.007098169066011906, 0.01190626

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.